In [37]:
# import numpy as np
# import pandas as pd
# from sklearn.metrics.pairwise import euclidean_distances
# from sklearn.metrics.pairwise import linear_kernel
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.feature_extraction.text import TfidfVectorizer
# from collections import Counter
# import mysql.connector
# import warnings
# warnings.filterwarnings("ignore")



# # join and concatenate inputted columns
# def create_soup(x, features):
#     soup_parts = [str(x[feature]) for feature in features if x[feature] is not None]  # Convert to string and filter out None values
#     return ' '.join(soup_parts)

# # count number of likeable elements for film
# def count_likeable(row):
#     features = ['primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
#     atts = sum(1 for col in row[features] if col is not None)
#     cast = len(row['cast'].split(','))
#     return atts+cast

# # get user loved films from db
# def get_loved_films(user_id):

#     mydb = mysql.connector.connect(
#         host="localhost",
#         user="root",
#         password="Leicester69lol",
#         database="users"
#     )

#     mycursor = mydb.cursor()

#     sql_query = "SELECT tconst FROM user_loved_films WHERE user_id = %s"

#     mycursor.execute(sql_query, (user_id,))

#     rows = mycursor.fetchall()

#     tconst_list = [tconst[0] for tconst in rows]

#     mycursor.close()
#     mydb.close()

#     loved_films_df = data[data['tconst'].isin(tconst_list)]

#     return loved_films_df

# # get user liked attributes from db
# def get_liked_attributes(user_id):

#     mydb = mysql.connector.connect(
#         host="localhost",
#         user="root",
#         password="Leicester69lol",
#         database="users"
#     )

#     mycursor = mydb.cursor()

#     sql_query = "SELECT * FROM user_liked_attributes WHERE user_id = %s"

#     mycursor.execute(sql_query, (user_id,))

#     attribute_fetch = mycursor.fetchall()

#     tconst_list = [row[1] for row in attribute_fetch]
#     attribute_bin = [row[1:] for row in attribute_fetch]

#     mycursor.close()
#     mydb.close()

#     attributes_template = ['tconst','primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
#     liked_attributes_df = pd.DataFrame(columns=attributes_template)

#     x = 0
#     for tconst, att_bin in zip(tconst_list, attribute_bin):
#         row_values = [tconst]  # Initialize row values with tconst
#         for liked, attribute in zip(att_bin[1:], attributes_template[1:]):
#             film_att = data[data['tconst'] == tconst]

#             if liked == 1:
#                 row_values.append(film_att[attribute].values[0])
#             else:
#                 row_values.append(None) 
#         liked_attributes_df.loc[x] = row_values
#         x+=1

#     return liked_attributes_df

# # get user liked cast from db
# def get_liked_cast(user_id):
#     mydb = mysql.connector.connect(
#         host="localhost",
#         user="root",
#         password="Leicester69lol",
#         database="users"
#     )

#     mycursor = mydb.cursor()

#     sql_query = "SELECT * FROM user_liked_cast WHERE user_id = %s"

#     mycursor.execute(sql_query, (user_id,))

#     cast_fetch = mycursor.fetchall()

#     mycursor.close()
#     mydb.close()

#     cast = [row[1:] for row in cast_fetch]
#     liked_cast_df = pd.DataFrame(cast, columns=['tconst', 'name'])

#     return liked_cast_df

# # get user watchlist from db
# def get_watchlist(user_id):
#     mydb = mysql.connector.connect(
#         host="localhost",
#         user="root",
#         password="Leicester69lol",
#         database="users"
#     )

#     mycursor = mydb.cursor()

#     sql_query = "SELECT * FROM user_watchlist WHERE user_id = %s"

#     mycursor.execute(sql_query, (user_id,))


#     watchlist_fetch = mycursor.fetchall()

#     mycursor.close()
#     mydb.close()

#     tconst_list = [row[1] for row in watchlist_fetch]
#     attributes_template = ['tconst','primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'cast', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
#     watchlist_df = pd.DataFrame(columns=attributes_template)

#     x = 0
#     for tconst in tconst_list:
#         watchlist_df.loc[x] = data[data['tconst'] == tconst][attributes_template].values[0]
#         x+=1

#     return watchlist_df

# # calculate % of liked attributes of film in profile
# def calculate_percentage(row):
#     total_attributes = row['total_likeable']
#     total_liked = row['num_liked_atts']
#     percentage = total_liked / total_attributes
#     return percentage 

# # count liked attributes for film
# def count_liked(row):
#     features = ['primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
#     atts = sum(1 for col in row[features] if pd.notna(col))  # Change this line
#     cast = 0
#     if not pd.isna(row['cast']):
#         cast = len(row['cast'].split(','))
    
#     return atts + cast

# # CREATE USER PROFILE
# def get_user_profile(user_id):
#     # get user film preferences from db
#     lovedFilms = get_loved_films(user_id)
#     likedAtt = get_liked_attributes(user_id)
#     likedCast = get_liked_cast(user_id)
#     watchlist = get_watchlist(user_id)

#     # merge liked cast with liked attribute table
#     likedCast_grouped = likedCast.groupby('tconst')['name'].apply(lambda x: ', '.join(x.dropna())).reset_index()
#     likedAttributes = pd.merge(likedAtt, likedCast_grouped, on='tconst', how='outer') #outer join instead of left?
#     likedAttributes.rename(columns={'name': 'cast'}, inplace=True)

#     # format user_profile dataframe
#     merged_love_liked = pd.concat([lovedFilms, likedAttributes], ignore_index=True)
#     user_profile = pd.merge(merged_love_liked, data[['tconst', 'total_likeable']], on='tconst', how='left')
#     user_profile = user_profile.drop(columns=['poster', 'total_likeable_x'])
#     user_profile.rename(columns={'total_likeable_y': 'total_likeable'}, inplace=True)

#     # initialise columns to calculate likeage
#     user_profile['num_liked_atts'] = 0
#     user_profile['likeage'] = 0.0
#     # count total liked attributes for each film in profile
#     for index, film in user_profile.iterrows():
#         tconst = film['tconst']
#         user_profile.loc[user_profile['tconst'] == tconst, 'num_liked_atts'] = count_liked(film)

#     # calculate likeage for each film in profile
#     user_profile['likeage'] = user_profile.apply(lambda row: calculate_percentage(row), axis=1)
#     user_profile.drop(columns=['num_liked_atts', 'total_likeable'], inplace=True)

#     return (user_profile, lovedFilms)

# # split user profile into likes by group
# def collate_liked_groups(user_profile):
#     group_dataframes = []

#     # Define dictionary to map group names to corresponding columns
#     group_columns = {
#         'liked_title_plot': ['primaryTitle', 'plot', 'likeage'],
#         'liked_cast': ['cast', 'likeage'],
#         'liked_crew': ['director', 'cinematographer', 'writer', 'producer', 'editor', 'composer', 'likeage'],
#         'liked_genre': ['genres', 'likeage'],
#         'liked_meta': ['averageRating', 'startYear', 'runtimeMinutes', 'likeage']
#     }

#     # Iterate over the dictionary and create group dataframes
#     for group_name, columns in group_columns.items():
#         group_df = user_profile[columns].copy()
#         # Drop rows with all NaN values in specified columns
#         group_df = group_df.dropna(subset=columns[:-1], how='all')
#         group_dataframes.append(group_df)

#     return group_dataframes

# # cosine similarity vector with tf-idf between films in row and column
# def create_similarity_vector(row, column):
#     tfidf = TfidfVectorizer(stop_words='english')
#     row_soup_temp = row.apply(lambda x: create_soup(x, row.columns), axis=1)
#     row_soup = row_soup_temp.fillna('')
#     row_matrix = tfidf.fit_transform(row_soup)

#     column_soup_temp = column.apply(lambda x: create_soup(x, column.columns), axis=1)
#     column_soup = column_soup_temp.fillna('')
#     column_matrix = tfidf.transform(column_soup)

#     return linear_kernel(row_matrix, column_matrix)

# # euclidean distance vector for numerical attributes of film
# def create_euclidean_vector(row, column):
#     scaler = MinMaxScaler()
#     row = row.fillna('')
#     column = column.fillna(0)

#     row_normalized = scaler.fit_transform(row)
#     column_normalized = scaler.transform(column)
#     distances = euclidean_distances(row_normalized, column_normalized)
#     euclidean_matrix = 1 / (1 + distances)
#     return euclidean_matrix

# # get top N films similar to user profile based on input vector (NOT CURRENTLY IN USE)
# def get_similar_films(vector, N):
#     filtered_vector = vector[~data['tconst'].isin(lovedFilms['tconst'])] #don't recommend film the user has already loved
#     mean_similarity = np.mean(filtered_vector, axis=1)
#     sorted_indices = np.argsort(mean_similarity)[::-1]
#     top_N = sorted_indices[:N]
#     return data.iloc[top_N]

# # function to calculate unified recommendations
# def get_unified_recommendations(user_profile_groups, similarity_vectors, N):

#     weighted_scores = {}
    
#     # scale similarity score in respective vector based on likeage of feature
#     for group, attributes in user_profile_groups.items():
#         similarity_vector = similarity_vectors[group][~data['tconst'].isin(user_profile['tconst'])] #don't recommend films already in profile
#         likeage_array = np.array(list(attributes['likeage'].tolist()))
#         weighted_similarity = similarity_vector * likeage_array

#         # meta columns have higher similarity since its numerical so adjust for this
#         if group == 'meta':
#             scaling_factor = 1 - weighted_similarity
#             weighted_similarity *= scaling_factor

#         # dictionary of groups and weighted similarity vectors 
#         weighted_scores[group] = weighted_similarity

#      # combine weighted similarity scores across all groups
#     combined_scores = np.sum(list(weighted_scores.values()), axis=0)

#     # calculate mean similarity scores
#     mean_similarity = np.mean(combined_scores, axis=1)

#     # sort the mean similarity scores and retrieve the top N indices
#     sorted_indices = np.argsort(mean_similarity)[::-1][:N]

#     return data.iloc[sorted_indices]


# data = pd.read_json('webpage/films.json')
# data['total_likeable'] = data.apply(lambda x: count_likeable(x), axis=1)
# data.head()


# # get update user profile and loved films
# get_profile = get_user_profile('57')
# user_profile = get_profile[0]
# lovedFilms = get_profile[1]

# # get groups of liked attributes
# grouped_likes = collate_liked_groups(user_profile)
# liked_plot = grouped_likes[0]
# liked_cast = grouped_likes[1]
# liked_crew = grouped_likes[2]
# liked_genre = grouped_likes[3]
# liked_meta = grouped_likes[4]

# # Define the list of features excluding 'likeage'
# plot_features = [col for col in liked_plot.columns if col != 'likeage']
# crew_features = [col for col in liked_crew.columns if col != 'likeage']
# cast_features = [col for col in liked_cast.columns if col != 'likeage']
# genre_features = [col for col in liked_genre.columns if col != 'likeage']
# meta_features = [col for col in liked_meta.columns if col != 'likeage']

# # Call the function with the selected columns
# plot_matrix = create_similarity_vector(data[plot_features], liked_plot[plot_features])
# crew_matrix = create_similarity_vector(data[crew_features], liked_crew[crew_features])
# cast_matrix = create_similarity_vector(data[cast_features], liked_cast[cast_features])
# genre_matrix = create_similarity_vector(data[genre_features], liked_genre[genre_features])
# meta_matrix = create_euclidean_vector(data[meta_features], liked_meta[meta_features])


# # Similarity vectors for each group
# similarity_vectors = {
#     'plot': plot_matrix,
#     'cast': cast_matrix,
#     'crew': crew_matrix,
#     'genre': genre_matrix,
#     'meta': meta_matrix
# }

# user_profile_groups = {
#     'plot': liked_plot,
#     'cast': liked_cast,
#     'crew': liked_crew,
#     'genre': liked_genre,
#     'meta': liked_meta
# }


# get_unified_recommendations(user_profile_groups, similarity_vectors, 20)



In [38]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import linear_kernel
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import mysql.connector
import warnings
warnings.filterwarnings("ignore")

data = pd.read_json('webpage/films.json')

# join and concatenate inputted columns
def create_soup(x, features):
    soup_parts = [str(x[feature]) for feature in features if x[feature] is not None]  # Convert to string and filter out None values
    return ' '.join(soup_parts)

# count number of likeable elements for film
def count_likeable(row):
    features = ['primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
    atts = sum(1 for col in row[features] if col is not None)
    cast = len(row['cast'].split(','))
    return atts+cast

# get user loved films from db
def get_loved_films(user_id):

    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Leicester69lol",
        database="users"
    )

    mycursor = mydb.cursor()

    sql_query = "SELECT tconst FROM user_loved_films WHERE user_id = %s"

    mycursor.execute(sql_query, (user_id,))

    rows = mycursor.fetchall()

    tconst_list = [tconst[0] for tconst in rows]

    mycursor.close()
    mydb.close()

    loved_films_df = data[data['tconst'].isin(tconst_list)]

    return loved_films_df

# get user liked attributes from db
def get_liked_attributes(user_id):

    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Leicester69lol",
        database="users"
    )

    mycursor = mydb.cursor()

    sql_query = "SELECT * FROM user_liked_attributes WHERE user_id = %s"

    mycursor.execute(sql_query, (user_id,))

    attribute_fetch = mycursor.fetchall()

    tconst_list = [row[1] for row in attribute_fetch]
    attribute_bin = [row[1:] for row in attribute_fetch]

    mycursor.close()
    mydb.close()

    attributes_template = ['tconst','primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
    liked_attributes_df = pd.DataFrame(columns=attributes_template)

    x = 0
    for tconst, att_bin in zip(tconst_list, attribute_bin):
        row_values = [tconst]  # Initialize row values with tconst
        for liked, attribute in zip(att_bin[1:], attributes_template[1:]):
            film_att = data[data['tconst'] == tconst]

            if liked == 1:
                row_values.append(film_att[attribute].values[0])
            else:
                row_values.append(None) 
        liked_attributes_df.loc[x] = row_values
        x+=1

    return liked_attributes_df

# get user liked cast from db
def get_liked_cast(user_id):
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Leicester69lol",
        database="users"
    )

    mycursor = mydb.cursor()

    sql_query = "SELECT * FROM user_liked_cast WHERE user_id = %s"

    mycursor.execute(sql_query, (user_id,))

    cast_fetch = mycursor.fetchall()

    mycursor.close()
    mydb.close()

    cast = [row[1:] for row in cast_fetch]
    liked_cast_df = pd.DataFrame(cast, columns=['tconst', 'name'])

    return liked_cast_df

# get user watchlist from db
def get_watchlist(user_id):
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Leicester69lol",
        database="users"
    )

    mycursor = mydb.cursor()

    sql_query = "SELECT * FROM user_watchlist WHERE user_id = %s"

    mycursor.execute(sql_query, (user_id,))


    watchlist_fetch = mycursor.fetchall()

    mycursor.close()
    mydb.close()

    tconst_list = [row[1] for row in watchlist_fetch]
    attributes_template = ['tconst','primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'cast', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
    watchlist_df = pd.DataFrame(columns=attributes_template)

    x = 0
    for tconst in tconst_list:
        watchlist_df.loc[x] = data[data['tconst'] == tconst][attributes_template].values[0]
        x+=1

    return watchlist_df

# calculate % of liked attributes of film in profile
def calculate_percentage(row):
    total_attributes = row['total_likeable']
    total_liked = row['num_liked_atts']
    percentage = total_liked / total_attributes
    return percentage 

# count liked attributes for film
def count_liked(row):
    features = ['primaryTitle', 'plot', 'averageRating', 'genres', 'runtimeMinutes', 'startYear', 'director', 'cinematographer', 'writer', 'producer', 'editor', 'composer']
    atts = sum(1 for col in row[features] if pd.notna(col))  # Change this line
    cast = 0
    if not pd.isna(row['cast']):
        cast = len(row['cast'].split(','))
    
    return atts + cast

# CREATE USER PROFILE
def get_user_profile(user_id):
    # get user film preferences from db
    lovedFilms = get_loved_films(user_id)
    likedAtt = get_liked_attributes(user_id)
    likedCast = get_liked_cast(user_id)
    watchlist = get_watchlist(user_id)

    # merge liked cast with liked attribute table
    likedCast_grouped = likedCast.groupby('tconst')['name'].apply(lambda x: ', '.join(x.dropna())).reset_index()
    likedAttributes = pd.merge(likedAtt, likedCast_grouped, on='tconst', how='outer') #outer join instead of left?
    likedAttributes.rename(columns={'name': 'cast'}, inplace=True)

    # format user_profile dataframe
    merged_love_liked = pd.concat([lovedFilms, likedAttributes], ignore_index=True)
    user_profile = pd.merge(merged_love_liked, data[['tconst', 'total_likeable']], on='tconst', how='left')
    user_profile = user_profile.drop(columns=['poster', 'total_likeable_x'])
    user_profile.rename(columns={'total_likeable_y': 'total_likeable'}, inplace=True)

    # initialise columns to calculate likeage
    user_profile['num_liked_atts'] = 0
    user_profile['likeage'] = 0.0
    # count total liked attributes for each film in profile
    for index, film in user_profile.iterrows():
        tconst = film['tconst']
        user_profile.loc[user_profile['tconst'] == tconst, 'num_liked_atts'] = count_liked(film)

    # calculate likeage for each film in profile
    user_profile['likeage'] = user_profile.apply(lambda row: calculate_percentage(row), axis=1)
    user_profile.drop(columns=['num_liked_atts', 'total_likeable'], inplace=True)

    return (user_profile, lovedFilms)

# split user profile into likes by group
def collate_liked_groups(user_profile):
    group_dataframes = []

    # Define dictionary to map group names to corresponding columns
    group_columns = {
        'liked_title_plot': ['primaryTitle', 'plot', 'likeage'],
        'liked_cast': ['cast', 'likeage'],
        'liked_crew': ['director', 'cinematographer', 'writer', 'producer', 'editor', 'composer', 'likeage'],
        'liked_genre': ['genres', 'likeage'],
        'liked_meta': ['averageRating', 'startYear', 'runtimeMinutes', 'likeage']
    }

    # Iterate over the dictionary and create group dataframes
    for group_name, columns in group_columns.items():
        group_df = user_profile[columns].copy()
        # Drop rows with all NaN values in specified columns
        # group_df = group_df.dropna(subset=columns[:-1], how='all')
        group_dataframes.append(group_df)

    return group_dataframes

# cosine similarity vector with tf-idf between films in row and column
def create_similarity_vector(row, column):
    tfidf = TfidfVectorizer(stop_words='english')
    row_soup_temp = row.apply(lambda x: create_soup(x, row.columns), axis=1)
    row_soup = row_soup_temp.fillna('')
    row_matrix = tfidf.fit_transform(row_soup)

    column_soup_temp = column.apply(lambda x: create_soup(x, column.columns), axis=1)
    column_soup = column_soup_temp.fillna('')
    column_matrix = tfidf.transform(column_soup)

    return linear_kernel(row_matrix, column_matrix)

# euclidean distance vector for numerical attributes of film
def create_euclidean_vector(row, column):
    scaler = MinMaxScaler()
    row = row.fillna('')
    column = column.fillna(0)

    row_normalized = scaler.fit_transform(row)
    column_normalized = scaler.transform(column)
    distances = euclidean_distances(row_normalized, column_normalized)
    euclidean_matrix = 1 / (1 + distances)
    return euclidean_matrix

# get top N films similar to user profile based on input vector (NOT CURRENTLY IN USE)
def get_similar_films(vector, N):
    # filtered_vector = vector[~data['tconst'].isin(lovedFilms['tconst'])] #don't recommend film the user has already loved
    mean_similarity = np.mean(vector, axis=1)
    sorted_indices = np.argsort(mean_similarity)[::-1]
    top_N = sorted_indices[:N]
    return data.iloc[top_N]


data['total_likeable'] = data.apply(lambda x: count_likeable(x), axis=1)


In [39]:
get_user_profile('57')[0]

,tconst,primaryTitle,plot,averageRating,genres,runtimeMinutes,startYear,cast,director,cinematographer,writer,producer,editor,composer,likeage
0,tt0848228,The Avengers,When an unexpected enemy emerges and threatens...,8.0,"Action,Sci-Fi",143,2012,"Robert Downey Jr., Chris Evans, Jeremy Renner,...",Joss Whedon,Seamus McGarvey,Zak Penn,Kevin Feige,Jeffrey Ford,Alan Silvestri,1.0


In [40]:
# function to calculate unified recommendations
def get_unified_recommendations(user_profile_groups, similarity_vectors, exclude, N):

    weighted_scores = {}
    
    # scale similarity score in respective vector based on likeage of feature
    for group, attributes in user_profile_groups.items():
        similarity_vector = similarity_vectors[group] 
        likeage_array = np.array(list(attributes['likeage'].tolist()))
        weighted_similarity = similarity_vector * likeage_array
  
        # meta columns have higher similarity since its numerical so adjust for this
        if group == 'meta':
            scaling_factor = 1 - weighted_similarity
            weighted_similarity *= scaling_factor

        # dictionary of groups and weighted similarity vectors 
        weighted_scores[group] = weighted_similarity

     # combine weighted similarity scores across all groups
    combined_scores = np.sum(list(weighted_scores.values()), axis=0)

    # calculate mean similarity scores
    mean_similarity = np.mean(combined_scores, axis=1)

    sorted_indices = np.argsort(mean_similarity)[::-1][:N]

    sorted_films = data.iloc[sorted_indices]

    filtered_recommendations = sorted_films[~sorted_films['tconst'].isin(exclude['tconst'])]

    return filtered_recommendations


# get update user profile and loved films
get_profile = get_user_profile('57')
user_profile = get_profile[0]
lovedFilms = get_profile[1]

# get groups of liked attributes
grouped_likes = collate_liked_groups(user_profile)
liked_plot = grouped_likes[0]
liked_cast = grouped_likes[1]
liked_crew = grouped_likes[2]
liked_genre = grouped_likes[3]
liked_meta = grouped_likes[4]

# Define the list of features excluding 'likeage'
plot_features = [col for col in liked_plot.columns if col != 'likeage']
crew_features = [col for col in liked_crew.columns if col != 'likeage']
cast_features = [col for col in liked_cast.columns if col != 'likeage']
genre_features = [col for col in liked_genre.columns if col != 'likeage']
meta_features = [col for col in liked_meta.columns if col != 'likeage']

# Call the function with the selected columns
plot_matrix = create_similarity_vector(data[plot_features], liked_plot[plot_features])
crew_matrix = create_similarity_vector(data[crew_features], liked_crew[crew_features])
cast_matrix = create_similarity_vector(data[cast_features], liked_cast[cast_features])
genre_matrix = create_similarity_vector(data[genre_features], liked_genre[genre_features])
meta_matrix = create_euclidean_vector(data[meta_features], liked_meta[meta_features])

# Similarity vectors for each group
similarity_vectors = {
    'plot': plot_matrix,
    'cast': cast_matrix,
    'crew': crew_matrix,
    'genre': genre_matrix,
    'meta': meta_matrix
}

# meta_matrix
user_profile_groups = {
    'plot': liked_plot,
    'cast': liked_cast,
    'crew': liked_crew,
    'genre': liked_genre,
    'meta': liked_meta
}

user_profile
# cast_matrix
# get_unified_recommendations(user_profile_groups, similarity_vectors, lovedFilms, 10)


,tconst,primaryTitle,plot,averageRating,genres,runtimeMinutes,startYear,cast,director,cinematographer,writer,producer,editor,composer,likeage
0,tt0848228,The Avengers,When an unexpected enemy emerges and threatens...,8.0,"Action,Sci-Fi",143,2012,"Robert Downey Jr., Chris Evans, Jeremy Renner,...",Joss Whedon,Seamus McGarvey,Zak Penn,Kevin Feige,Jeffrey Ford,Alan Silvestri,1.0


In [41]:
get_similar_films(plot_matrix, 5)['plot']

233      When an unexpected enemy emerges and threatens...
20686    When an international terrorist cult threatens...
18099    A veteran US Secret Service agent must thwart ...
5086     While trying to remain a good husband and fath...
14239    He Is My Enemy, Partner and Father-In-Law is a...
Name: plot, dtype: object

In [42]:
get_similar_films(cast_matrix, 5)


,tconst,primaryTitle,plot,averageRating,genres,runtimeMinutes,startYear,cast,director,cinematographer,writer,producer,editor,composer,poster,total_likeable
233,tt0848228,The Avengers,When an unexpected enemy emerges and threatens...,8.0,"Action,Sci-Fi",143,2012,"Robert Downey Jr., Chris Evans, Jeremy Renner,...",Joss Whedon,Seamus McGarvey,Zak Penn,Kevin Feige,Jeffrey Ford,Alan Silvestri,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,16
12678,tt0314676,The Singing Detective,"From his hospital bed, a writer suffering from...",5.4,"Comedy,Crime,Musical",109,2003,"Robert Downey Jr., Mel Gibson, Jeremy Northam,...",Keith Gordon,Tom Richmond,Dennis Potter,"Bruce Davey, Steven Haft",Jeff Wishengrad,None,/bjkLSHpV294HIllRpHKWkNICs3N.jpg,15
7971,tt0335266,Lost in Translation,"Two lost souls visiting Tokyo -- the young, ne...",7.7,"Comedy,Drama",102,2003,"Bill Murray, Giovanni Ribisi, Scarlett Johanss...",Sofia Coppola,Lance Acord,None,Ross Katz,Sarah Flack,Kevin Shields,/4k4Yz08WGfbu8ITIjaG99XTeco8.jpg,15
19365,tt14230388,Asteroid City,"In an American desert town circa 1955, the iti...",6.5,"Comedy,Drama,Romance",105,2023,"Jason Schwartzman, Tom Hanks, Jeffrey Wright, ...",Wes Anderson,Robert D. Yeoman,Roman Coppola,"Jeremy Dawson, Steven Rales",None,Alexandre Desplat,/qdq40gRS8xKnpFt5V75t6lUHgpx.jpg,15
13215,tt0113321,Home for the Holidays,"After losing her job, making out with her soon...",6.6,"Comedy,Drama,Romance",103,1995,"Robert Downey Jr., Charles Durning, Holly Hunt...",Jodie Foster,Lajos Koltai,"Chris Radant, W.D. Richter",Peggy Rajski,None,Mark Isham,/3zqlKdOttWyExzopVxglz50Vjxq.jpg,15


In [21]:
get_similar_films(crew_matrix, 5)


,tconst,primaryTitle,plot,averageRating,genres,runtimeMinutes,startYear,cast,director,cinematographer,writer,producer,editor,composer,poster,total_likeable
18620,tt0051600,Fanfare,"In the small Dutch village of Lagerheide, two ...",7.3,"Comedy,Music",86,1958,"Hans Kaart, Bernard Droog, Wim Van den Heuvel,...",Bert Haanstra,Eduard van der Enden,Jan Blokker,Rudolf Meyer,Ralph Sheldon,Jan Mul,/5znrKi2sicXgr7RH51Unqk43zx1.jpg,16
248,tt8019694,Frankie,About three generations of a family grappling ...,5.4,"Drama,Romance",100,2019,"Ariyon Bakare, Isabelle Huppert, Sennia Nanua,...",Ira Sachs,Rui Poças,Mauricio Zacharias,"Saïd Ben Saïd, Michel Merkt",None,Dickon Hinchliffe,/sbfrPcVHhKU6sPUP7ZVPsy7li4o.jpg,15
6438,tt0105220,Decay,"1986, Chernobyl disaster. Couples, friends, an...",6.6,Drama,103,1990,"Sergey Shakurov, Stanislav Stankevich, Georgiy...",Mikhail Belikov,Vasiliy Trushkovskiy,Oleg Prihodko,Mikhail Kostyukovskiy,None,"Igor Stetsyuk, Vladimir Vysotskiy",/3Yq0CsEzbCCbk03V5fCsD0lW7vD.jpg,15
12947,tt0081496,Dolaon solimsa jubangjang,"Ko Lin kills the brother of Hung Lau, an outla...",5.7,"Action,Comedy",84,1980,"Jin-Hwa Jeong, Ryong Wang, Un Bi, Noo-lee Jin",Jeong-yong Kim,Il-man Jeong,Seok-hun Yun,Yong-deok Kim,Dong-Chun Hyeon,Cheol-hyeok Lee,/eiW7g32lk9lWokAEFSTO3jdOmg1.jpg,16
17484,tt0184526,Gorgeous,"When Ah Bu, a girl from a small fishing town i...",6.0,"Action,Comedy,Romance",121,1999,"Jackie Chan, Tony Leung Chiu-wai, Emil Chau, S...",Vincent Kok,Man-Po Cheung,"Ivy Ho, Yiu Fai Lo",Raymond Chow,None,Dan-yee Wong,/vzY0zAdfnLZaVQrQcivasBDTmm4.jpg,15


In [25]:
get_similar_films(genre_matrix, 5)


,tconst,primaryTitle,plot,averageRating,genres,runtimeMinutes,startYear,cast,director,cinematographer,writer,producer,editor,composer,poster,total_likeable
18650,tt23490532,Nachindi Girl Friendu,"Raja Ram aka Raja, a confident youngster who h...",5.3,"Action,Thriller",130,2022,"Sairaj Suthari, Srikanth Iyengar, Gayatri Bhar...",Guru Pawan,"Siddam Manohar, Siddam Naresh",None,Atluri Narayana Rao,Kala Sagar Udagandla,Gifton Elias,/kOyQqVenAMDjzXxOCYAVrMF4Cjh.jpg,15
17329,tt0419979,Alter Ego,Three aspiring young Japanese models are on th...,4.7,Horror,63,2002,"Hideo Sakaki, Nobuko Sakuma, Sena, Chieko Kawabe",Issei Shibata,Shin Hayasaka,None,Yukihiko Yamaguchi,Etsuko Kimura,Saku Sakamoto,/gum4xP07ZU1EkvpsExpCxisn44g.jpg,15
10041,tt11318086,The Fall of the Queens,"One mother, two sisters that make each other j...",5.2,Drama,83,2021,"Franco Rizzaro, Malena Filmus, Lola Abraldes, ...",Lucas Nazareno Turturro,Nicolas Trovato,"Constanza Boquet, Mauro Guevara",Victoria Aizenstat,None,Sebastián Escofet,/fv2z4hOFGQsZUci3NNGDwHKXxZH.jpg,15
1959,tt0420724,A Killer Without a Grave,Nakata (Keiichiro Akagi) is a nihilistic young...,6.7,"Action,Crime",87,1961,"Gorô Mutsumi, Keiichirô Akagi, Reiko Sasamori,...",Yôichi Ushihara,Shinsaku Himeda,Toshio Matsuura,None,Masanori Tsujii,Taiichirô Kosugi,/uuIVJo1dRGa8TVOK9QDOwtwmidz.jpg,16
10836,tt0201046,Diamond Safari,"Raphaël, a former paratrooper, is unable to ad...",6.0,"Crime,Drama,Romance",90,1966,"Jean-Louis Trintignant, Horst Frank, Hellmut L...",Michel Drach,Andréas Winding,Albert Kantof,None,Geneviève Winding,Ward Swingle,/vbhYov4nrp5nrUkSrDSSxzF2kTU.jpg,15


In [14]:
get_similar_films(meta_matrix, 5)


,tconst,primaryTitle,plot,averageRating,genres,runtimeMinutes,startYear,cast,director,cinematographer,writer,producer,editor,composer,poster,total_likeable
233,tt0848228,The Avengers,When an unexpected enemy emerges and threatens...,8.0,"Action,Sci-Fi",143,2012,"Robert Downey Jr., Chris Evans, Jeremy Renner,...",Joss Whedon,Seamus McGarvey,Zak Penn,Kevin Feige,Jeffrey Ford,Alan Silvestri,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,16
2027,tt2576852,The Tale of The Princess Kaguya,Found inside a shining stalk of bamboo by an o...,8.0,"Animation,Drama,Family",137,2013,"James Caan, James Marsden, Chloë Grace Moretz,...",Isao Takahata,Keisuke Nakamura,Riko Sakaguchi,"Yoshiaki Nishimura, Geoffrey Wexler",None,Joe Hisaishi,/cQidJuA546OSSXKWXoiCeINDxuj.jpg,15
14647,tt1291584,Warrior,The youngest son of an alcoholic former boxer ...,8.1,"Action,Drama,Sport",140,2011,"Tom Hardy, Nick Nolte, Joel Edgerton, Jennifer...",Gavin O'Connor,Masanobu Takayanagi,"Anthony Tambakis, Cliff Dorfman",Greg O'Connor,None,Mark Isham,/iM8n4nZJPR2abpnyZ36FUgHiRjr.jpg,15
22076,tt3051894,Cloud Capped Star,"Nilkantha, an artist, is admitted in a mental ...",8.1,"Biography,Drama",154,2013,"Saswata Chatterjee, Abir Chatterjee, Rahul Ban...",Kamaleswar Mukherjee,Soumik Haldar,None,"Shrikant Mohta, Mahendra Soni",Rabiranjan Maitra,Debojyoti Mishra,/1oftM3FelslyYenutcwd4u5lcLE.jpg,15
15626,tt2585562,Pizza,A pizza delivery boy lands in a mysterious cir...,7.9,"Horror,Thriller",127,2012,"Vijay Sethupathi, Aadukalam Naren, Karunakaran...",Karthik Subbaraj,Gopi Amarnath,Prasath Ramar,C.V. Kumar,Leo John Paul,Santhosh Narayanan,/ojC9NacZhfSwUn9aAMXUtDLV52e.jpg,16


In [15]:
plot_matrix

array([[0.        ],
       [0.02689671],
       [0.        ],
       ...,
       [0.00879722],
       [0.        ],
       [0.        ]])

In [16]:
cast_matrix

array([[0.        ],
       [0.05134752],
       [0.        ],
       ...,
       [0.        ],
       [0.        ],
       [0.        ]])

In [17]:
crew_matrix

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [18]:
genre_matrix

array([[0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       ...,
       [0.        , 0.47214275],
       [0.        , 0.43100091],
       [0.27198253, 0.        ]])

In [19]:
meta_matrix

array([[0.84691965],
       [0.74443825],
       [0.80103622],
       ...,
       [0.83037339],
       [0.61944456],
       [0.6930204 ]])